**Encoding Part**
![image](../images/self_attention.png)
We have an input sequence and a target sequence of length 8 (8 tokens). For the training, we take the batch size as 64 and model dimension as 512. So the first batch input will be of shape (64,8,512). To this input we add positional embeddings and the output will be of same dimension

Now comes the first encoder which takes the (64,8,512) input. We have 8 tokens, so we need to get a new representation for this vector by taking some weighted aggregation of other tokens. Now we make 3 copies of input(queries,keys,values).

![image](../images/transformer_architecuture.png) 
- Queries are $q_1,q_2,q_3,...,q_8$ each query has  dimension $d_q$.
- Values  are $v_1,v_2,...,v_8$ each value has dimension $d_v$.
- Keys are $k_1,k_2,... ,k_8$ each key has dimension $d_k$.
- $T = 8; d_q;d_k;d_v,d_m$
- $Q=(B,T,d_q) ; K=(B,T,d_k); V = (B,T,d_v)$

We compute the attention weights as follows; 

$$
Attention(Q,K)=softmax(\frac{QK^T}{\frac{1}{\sqrt{d_k}}})
$$

Attention weight matrix will be of shape $(T,T)$. This weight will be multiplied with the value vectors. To get all the token representation. After multiplied with the value vector $V$, the output will be of shape $(B,T,d_v)$. 

This output will pass through multiple encoder layers and finally a full connected layer, and the final output will of shape $(B,T,d_m)$. 

**Intuition of Self-Attention** 
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `weights` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

**Decoding part**
In the decoder part, we have a masked self attention layer first. Remember, in the training stage, decoder predicts the outputs all at ones using masked attention. But in inference or testing stage it predicts the outputs one by one. In masked attention, each token attention vector is created by considering all the tokens from the past. The steps in decoder part are token embedding, then positional embedding, addition of both, masked multi-headed self attention, encoder-decoder multi-headed cross attention, linear and finally the softmax layer. 

We take the target sequence first, then we apply positional embedding. 

Then it creates a masked self attention matrix. We take this output as query matrix for the next multi-headed attention layers.

Finally the ouputs will pass through linear and softmax layers.



**Inference Process** 
* At first time step one we have only one token 
* decoder_start_token_shape = $(1,128)$ 
* creating three copies ( for masked self attention)
* multiplying $q.k^T => (1,1)$ -> a single weight vector
* multiplying with the value vectors and getting * aggregated measure 
* $weight* values => (1,128)$ 
* This is the query for the next layer* (encoder-decoder cross attention)
* from encoder we got keys and values of shape (1,7,* 128) -> 7 is the number of tokens 
* $(1,1,128)*(1,128,7) => (1,1,7)$ -> 7 weights of the * input query with all the keys in the encoder key * matrix 
* multiplying $(1,1,7)*values$ 
* $(1,1,7)*(1,7,128) => (1,1,128)$ 
* This output will pass through linear layer which * takes a tensor of feature dimension 128 
* linear layer will output the final output * dimensions 
* At the second time step we have two tokens (sos * token and the first decoder output which is * predicted)

* So the input will be of shape $(1,2,128)$ 
* self attention output will be again $(1,2,128)$

